In [1]:
import numpy                as np
from os                     import listdir
import matplotlib.pyplot    as plt
import matplotlib           as mpl
import matplotlib.lines     as mlines
from natsort                import natsorted
import os
import json

from matplotlib             import rcParams, rc
rcParams.update({'figure.dpi': 200})
rc('font', family='serif')
rc('text', usetex=True)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('/STER/silkem/MACE/')
import src.mace.utils as utils
import src.mace.CSE_0D.dataset as ds 

from matplotlib             import rcParams, rc
rcParams.update({'figure.dpi': 200})


def makeOutputDir(path):
    if not os.path.exists(path):
        os.mkdir(path)
    return path

specs_dict, idx_specs = utils.get_specs()


def get_time(dt, idx):
    t = np.zeros(len(dt))
    t[0] = CSEmodel.time[idx]
    for i in range(0,len(t)-1):
        t[i+1] = (dt[i]+t[i])
        
    return np.array(t)

In [ ]:
path = ''

CSEmod = ds.CSEmod(path)